<a href="https://colab.research.google.com/github/mjevans26/Satellite_ComputerVision/blob/master/UNET_G4G_2019_solar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#@title Author: Michael Evans { display-mode: "form" }
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Introduction

This notebook demonstrates methods used to delineate all of the ground-mounted solar arrays in North Carolina using free satellite imagery.  Our workflow generates and exports satellite imagery data from Google Earth Engine for analysis in Tensorflow.  This analysis predicts the probability of the presence of a solar array as a function of the visible, infrared, and near infrared bands in Sentinel-2 imagery.  The model is a [fully convolutional neural network (FCNN)](https://www.cv-foundation.org/openaccess/content_cvpr_2015/papers/Long_Fully_Convolutional_Networks_2015_CVPR_paper.pdf), specifically [U-net](https://arxiv.org/abs/1505.04597).  This relatively simple model is a mostly unmodified version of [this example](https://github.com/tensorflow/models/blob/master/samples/outreach/blogs/segmentation_blogpost/image_segmentation.ipynb) from the TensorFlow docs.  This notebook shows:

1.   Exporting training/testing patches from Earth Engine, suitable for training an FCNN model.
2.   Preprocessing.
3.   Training and validating an FCNN model.
4.   Making predictions with the trained model and importing them to Earth Engine.

# Setup software libraries

Install needed libraries to the notebook VM.  Authenticate as necessary.

In [0]:
# Cloud authentication.
from google.colab import auth
auth.authenticate_user()

In [0]:
# Earth Engine install to notebook VM.
!pip install earthengine-api

In [0]:
# Import, authenticate and initialize the Earth Engine library.
import ee
ee.Authenticate()
ee.Initialize()

In [0]:
# Tensorflow setup.
import tensorflow as tf

tf.enable_eager_execution()
print(tf.__version__)

%load_ext tensorboard

In [0]:
# Folium setup.
import folium
print(folium.__version__)

# Define a method for displaying Earth Engine image tiles to a folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = "Map Data © Google Earth Engine",
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Define the URL format used for Earth Engine generated map tiles.
#EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'

##Mount Google Drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# move into repo directory containing modules we need
%cd '/content/drive/My Drive/repos/Satellite_ComputerVision'

# Variables

Declare the variables that will be in use throughout the notebook.

## Specify your Cloud Storage Bucket
Specify a cloud storage bucket to which you have read/write access

In [0]:
BUCKET = 'cvod-203614-mlengine'

## Set other global variables

In [0]:
# Specify names locations for outputs in Cloud Storage. 
FOLDER = 'NC_solar'
PRED_BASE = 'data/predict'
TRAINING_BASE = 'data/training'
EVAL_BASE = 'data/eval'
MODEL_BASE = 'models/UNET256'
log_dir = 'drive/My Drive/Tensorflow/models/UNET256'

# Specify inputs (Sentinel bands) to the model and the response variable.
opticalBands = ['B2', 'B3', 'B4']
thermalBands = ['B8', 'B11', 'B12']
# We may want to run some experiments where we use pca components
pcaBands = ['pc1', 'pc2', 'pc3']
BANDS = opticalBands + thermalBands# + pcaBands
RESPONSE = 'landcover'
FEATURES = BANDS + [RESPONSE]
SCENEID = 'SENSING_ORBIT_NUMBER'

# Specify the size and shape of patches expected by the model.
KERNEL_SIZE = 256
KERNEL_SHAPE = [KERNEL_SIZE, KERNEL_SIZE]
COLUMNS = [
  tf.io.FixedLenFeature(shape=KERNEL_SHAPE, dtype=tf.float32) for k in FEATURES
]
FEATURES_DICT = dict(zip(FEATURES, COLUMNS))

# Sizes of the training and evaluation datasets.
TRAIN_SIZE = 7000
EVAL_SIZE = 3000

# Specify model training parameters.
BATCH_SIZE = 16
EPOCHS = 20
BUFFER_SIZE = 8000
OPTIMIZER = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
LOSS = 'binary_crossentropy'
METRICS = [tf.keras.metrics.categorical_accuracy, tf.keras.metrics.MeanIoU(num_classes=2)]

# Imagery

Process the imagery to use for predictor variables.  This is a three-month, cloud-free, Sentinel-2 composite corresponding to the latest date from which we have confirmed training data.  Display it in the notebook for a sanity check.

In [0]:
# add the Google Drive repo directory so we can use our modules
import sys
from os.path import join
sys.path.append('/content/drive/My Drive/repos/Satellite_ComputerVision/utils')

In [0]:
from clouds import basicQA
# Use Sentinel-2 surface reflectance data.
S2 = ee.ImageCollection("COPERNICUS/S2")
# Grab a feature corresponding to our study area - North Carolina
states = ee.FeatureCollection("TIGER/2016/States")
nc = states.filter(ee.Filter.eq('NAME', 'North Carolina'))
begin = '2019-01-01'
end = '2019-04-30'

# The image input collection is cloud-masked.
filtered = S2.filterDate(begin, end)\
.filterBounds(nc)\
.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))\
.map(basicQA)

# Create a simple median composite to visualize
image = filtered.median().select(BANDS).clip(nc)

# Use folium to visualize the imagery.
#mapid = image.getMapId({'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3})
rgbParams = {'bands': ['B4', 'B3', 'B2'],
             'min': 0,
             'max': 0.3}

nirParams = {'bands': ['B8', 'B11', 'B12'],
             'min': 0,
             'max': 0.3}

map = folium.Map(location=[35.402, -78.376])
map.add_ee_layer(image, rgbParams, 'Color')
map.add_ee_layer(image, nirParams, 'Thermal')

map.add_child(folium.LayerControl())
map

Prepare the response variable.  This is the footprints of ground mounted solar arrays as of 2016, coded into a background class [0] and a target class [1]. Display on the map to verify.

In [0]:
def set_landcover(ft):
  """
  Add a property to a feature and set it to 1
  Parameters:
    ft (ee.Feature): feature to have property added
  Returns:
    ee.Feature: input feature with 'label' property set to 1
  """
  return ft.set('label', 1)

# Get solar footprints data from our GEE Asset
NC_solar_footprints = ee.FeatureCollection("users/taylorminich/NC_solar_footprints")
# Label each polygon with property 'label' equal to 1
NC_solar_footprints = NC_solar_footprints.map(set_landcover)
# Create an image with all pixels equal to 0
blankimg = ee.Image.constant(0)
# Convert solar footprints to an image (band value will be 1 based on 'label')
solar_footprint = NC_solar_footprints.reduceToImage(['label'], ee.Reducer.first())
# Convert pixels of blank image to 1 where the values of the footprint image are 1
# and rename to 'landcover'
labelimg = blankimg.where(solar_footprint, solar_footprint).rename('landcover')

solarParams = {'bands': 'landcover', 'min':0, 'max': 1}

map = folium.Map(location = [35.402, -78.376])
map.add_ee_layer(labelimg,  solarParams, 'Solar footprint')
map.add_child(folium.LayerControl())
map

### Calibration
For consistency in predictive ability across contexts, we calibrate all images in the collection to a standard, then normalize the image bands to [0,1] after squashing extreme values where the sensor was likely 'washed out'

In [0]:
from calibration import equalize_collection

In [0]:
# calibrate all scenes in the collection using histogram equalization
equalized = equalize_collection(filtered, BANDS, SCENEID)

# need to cast all images in resulting collection to same type for 
normImage = equalized.cast(dict(zip(BANDS, ['float']*6)), BANDS).median()

Stack the normalized sentinel composite and binary solar indicator image to create a single image from which samples can be taken.  Convert the image into an array image in which each pixel stores 256x256 patches of pixels for each band.  This is a key step that bears emphasis: to export training patches, convert a multi-band image to [an array image](https://developers.google.com/earth-engine/arrays_array_images#array-images) using [`neighborhoodToArray()`](https://developers.google.com/earth-engine/api_docs#eeimageneighborhoodtoarray), then sample the image at points.

In [0]:
featureStack = ee.Image.cat([
  normImage.select(BANDS),
  labelimg.select(RESPONSE)
]).float()

list = ee.List.repeat(1, KERNEL_SIZE)
lists = ee.List.repeat(list, KERNEL_SIZE)
kernel = ee.Kernel.fixed(KERNEL_SIZE, KERNEL_SIZE, lists)

arrays = featureStack.neighborhoodToArray(kernel)

Use some pre-made geometries to sample the stack in strategic locations.  We constrain sampling to occur within 10km of mapped solar arrays. Because our target features are small and sparse, relative to the landscape, we also guide sampling based on their centroids to ensure that we get training data for solar arrays.

In [0]:
def buff(ft):
  return ft.buffer(10000)

def centroid(ft):
  return ft.centroid()

centroids = NC_solar_footprints.map(centroid)
studyArea = NC_solar_footprints.map(buff).union()
studyImage = ee.Image(0).byte().paint(studyArea, 1)
studyImage = studyImage.updateMask(studyImage)

aoiParams = {'min':0, 'max': 1, 'palette': ['red']}
map = folium.Map(location=[35.402, -78.376], zoom_start=8)
map.add_ee_layer(studyImage, aoiParams, 'Sampling area')
map.add_child(folium.LayerControl())
map

# Sampling

The mapped data look reasonable so take a sample from each polygon and merge the results into a single export.  The key step is sampling the array image at points, to get all the pixels in a 256x256 neighborhood at each point.  It's worth noting that to build the training and testing data for the FCNN, you export a single TFRecord file that contains patches of pixel values in each record.  You do NOT need to export each training/testing patch to a different image.  Since each record potentially contains a lot of data (especially with big patches or many input bands), some manual sharding of the computation is necessary to avoid the `computed value too large` error.  Specifically, the following code takes multiple (smaller) samples within each geometry, merging the results to get a single export.

In [0]:
!gsutil ls gs://cvod-203614-mlengine/NC_solar/data/

In [0]:
#@title Don't run
# Add a random column to the centroids
centroids = centroids.randomColumn('random')

# Define sample sizes for shards and chunks. 
# These numbers determined experimentally.
n = 100 # Number of shards in each chunk.
N = 1000 # Total sample size in each chunk.
C = 10# Number of chunks

# #First make sure we get samples from all the delineated examples
geomSample = ee.FeatureCollection([])
x = 0

while x < 1:
  region = centroids.filter(ee.Filter.rangeContains('random', x, x+0.05))
  print(region.size().getInfo())
  #get 256 patches from centroids in chunks of 10...?
  sample = arrays.sampleRegions(
      collection = region,
      #collection = centroidList.slice(x, x+10),
      scale = 10,
      properties = ['label'],
      tileScale = 12
  )
  x += 0.05
  geomSample = geomSample.merge(sample)
                                
geomSample = geomSample.randomColumn('random')
training = geomSample.filter(ee.Filter.gte('random', 0.3))
testing = geomSample.filter(ee.Filter.lt('random', 0.3))

desc = 'UNET_' + str(KERNEL_SIZE) + '_trainCent'
task = ee.batch.Export.table.toCloudStorage(
  collection = training,
  description = desc, 
  bucket = BUCKET, 
  fileNamePrefix = join(FOLDER, TRAINING_BASE, desc),
  fileFormat = 'TFRecord',
  selectors = BANDS + [RESPONSE]
)
task.start()

desc = 'UNET_' + str(KERNEL_SIZE) + '_evalCent'
task = ee.batch.Export.table.toCloudStorage(
  collection = testing,
  description = desc, 
  bucket = BUCKET, 
  fileNamePrefix = join(FOLDER, EVAL_BASE),
  fileFormat = 'TFRecord',
  selectors = BANDS + [RESPONSE]
)
task.start()

#now get a random sample of points from areas around mapped features

for c in range(C):
  geomSample = ee.FeatureCollection([])

  for i in range(n):
    sample = arrays.sample(
        region = studyArea,
        scale = 10,
        numPixels = N/n,
        seed = (c-i)+(i+c),
        tileScale = 8
    )
    geomSample = geomSample.merge(sample)

  #divide samples into training and evaluation data
  geomSample = geomSample.randomColumn('random')
  training = geomSample.filter(ee.Filter.gte('random', 0.3))
  testing = geomSample.filter(ee.Filter.lt('random', 0.3))

  desc = 'UNET_' + str(KERNEL_SIZE) + '_train'+str(c)
  task = ee.batch.Export.table.toCloudStorage(
    collection = training,
    description = desc, 
    bucket = BUCKET, 
    fileNamePrefix = join(FOLDER, TRAINING_BASE),
    fileFormat = 'TFRecord',
    selectors = BANDS + [RESPONSE]
  )
  task.start()

  desc = 'UNET_' + str(KERNEL_SIZE) + '_eval' + str(c)
  task = ee.batch.Export.table.toCloudStorage(
    collection = testing,
    description = desc, 
    bucket = BUCKET, 
    fileNamePrefix = join(FOLDER, EVAL_BASE),
    fileFormat = 'TFRecord',
    selectors = BANDS + [RESPONSE]
  )
  task.start() 

##Preprocessing
Define functions that apply random manipulations to our training data

In [0]:
def augColor(x):
    """Color augmentation

    Args:
        x: Image

    Returns:
        Augmented image
    """
    x = tf.image.random_hue(x, 0.08)
    x = tf.image.random_saturation(x, 0.6, 1.6)
    x = tf.image.random_brightness(x, 0.05)
    x = tf.image.random_contrast(x, 0.7, 1.3)
    return x
  
  
def augImg(img):
  outDims = tf.shape(img)[0:1]
  x = tf.image.random_flip_left_right(img)
  x = tf.image.random_flip_up_down(x)
  x = rotated = tf.image.rot90(x, tf.random_uniform(shape=[], minval=0, maxval=4, dtype=tf.int32))
  #x = zoom(x, outDims)
  #since were gonna map_fn this on a 4d image, output must be 3d, so squeeze the artificial 'sample' dimension
  return tf.squeeze(x)

def preprocess(img, labels):
  dims = tf.shape(img)
  #need to combine labels and bands for morphological transformations
  comb = tf.concat([img, tf.expand_dims(labels, axis = 2)], axis = 2)
  aug = aug_img(comb)
  #aug = tf.map_fn(fn = aug_img, elems = comb)
  labels = tf.squeeze(aug[:, :, -1])
  band_stack = color(aug[:, :, 0:dims[2]])
  return band_stack, labels

# Training data

Load the data exported from Earth Engine into a `tf.data.Dataset`.  The following are helper functions for that.

In [0]:
def parse_tfrecord(example_proto):
  """The parsing function.
  Read a serialized example into the structure defined by FEATURES_DICT.
  Args:
    example_proto: a serialized Example.
  Returns: 
    A dictionary of tensors, keyed by feature name.
  """
  return tf.io.parse_single_example(example_proto, FEATURES_DICT)


def to_tuple(inputs):
  """Function to convert a dictionary of tensors to a tuple of (inputs, outputs).
  Turn the tensors returned by parse_tfrecord into a stack in HWC shape.
  Args:
    inputs: A dictionary of tensors, keyed by feature name.
  Returns: 
    A dtuple of (inputs, outputs).
  """
  inputsList = [inputs.get(key) for key in FEATURES]
  stacked = tf.stack(inputsList, axis=0)
  # Convert from CHW to HWC
  stacked = tf.transpose(stacked, [1, 2, 0])
  stacked = augImg(stacked)
  return stacked[:,:,:len(BANDS)], stacked[:,:,len(BANDS):]


def get_dataset(pattern):
  """Function to read, parse and format to tuple a set of input tfrecord files.
  Get all the files matching the pattern, parse and convert to tuple.
  Args:
    pattern: A file pattern to match in a Cloud Storage bucket.
  Returns: 
    A tf.data.Dataset
  """
  glob = tf.gfile.Glob(pattern)
  dataset = tf.data.TFRecordDataset(glob, compression_type='GZIP')
  dataset = dataset.map(parse_tfrecord, num_parallel_calls=5)
  dataset = dataset.map(to_tuple, num_parallel_calls=5)
  return dataset

Use the helpers to read in the training dataset.  Print the first record to check.

In [0]:
def get_training_dataset():
	"""Get the preprocessed training dataset
  Returns: 
    A tf.data.Dataset of training data.
  """
	glob = 'gs://' + BUCKET + '/' + FOLDER + '/' + TRAINING_BASE + '/*'
	dataset = get_dataset(glob)
	dataset = dataset.shuffle(8000).batch(BATCH_SIZE).repeat()
	return dataset

training = get_training_dataset()

#check to make sure our records look like we expect
print(iter(training.take(1)).next())

# Evaluation data

Now do the same thing to get an evaluation dataset.  Note that unlike the training dataset, the evaluation dataset has a batch size of 1, is not repeated and is not shuffled.

In [0]:
def get_eval_dataset():
	"""Get the preprocessed evaluation dataset
  Returns: 
    A tf.data.Dataset of evaluation data.
  """
	glob = 'gs://' + BUCKET + '/' + FOLDER + '/' + EVAL_BASE + '/*'
	dataset = get_dataset(glob)
	dataset = dataset.batch(1).repeat()
	return dataset

evaluation = get_eval_dataset()

# Model

Here we use the Keras implementation of the U-Net model as found [in the TensorFlow examples](https://github.com/tensorflow/models/blob/master/samples/outreach/blogs/segmentation_blogpost/image_segmentation.ipynb).  The U-Net model takes 256x256 pixel patches as input and outputs per-pixel class probability, label or a continuous output.  We can implement the model essentially unmodified, but will use mean squared error loss on the sigmoidal output since we are treating this as a regression problem, rather than a classification problem.  Since impervious surface fraction is constrained to [0,1], with many values close to zero or one, a saturating activation function is suitable here.

##Metrics

We define a weighted binary cross entropy loss function because the training data is potentially sparse. This also gives us greater control over the rates of omission and commission prediciton errors. Because this is an image segmentation exercise, we may also be interested in the intersection over union as a loss measure.

In [0]:
from model import iou, weighted_bce
from tensorflow.python.keras import layers
from tensorflow.python.keras import losses
from tensorflow.python.keras import models
from tensorflow.python.keras import metrics
from tensorflow.python.keras import optimizers

def conv_block(input_tensor, num_filters):
	encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(input_tensor)
	encoder = layers.BatchNormalization()(encoder)
	encoder = layers.Activation('relu')(encoder)
	encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(encoder)
	encoder = layers.BatchNormalization()(encoder)
	encoder = layers.Activation('relu')(encoder)
	return encoder

def encoder_block(input_tensor, num_filters):
	encoder = conv_block(input_tensor, num_filters)
	encoder_pool = layers.MaxPooling2D((2, 2), strides=(2, 2))(encoder)
	return encoder_pool, encoder

def decoder_block(input_tensor, concat_tensor, num_filters):
	decoder = layers.Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding='same')(input_tensor)
	decoder = layers.concatenate([concat_tensor, decoder], axis=-1)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('relu')(decoder)
	decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('relu')(decoder)
	decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('relu')(decoder)
	return decoder

def get_model():
	inputs = layers.Input(shape=[None, None, len(BANDS)]) # 256
	encoder0_pool, encoder0 = encoder_block(inputs, 32) # 128
	encoder1_pool, encoder1 = encoder_block(encoder0_pool, 64) # 64
	encoder2_pool, encoder2 = encoder_block(encoder1_pool, 128) # 32
	encoder3_pool, encoder3 = encoder_block(encoder2_pool, 256) # 16
	encoder4_pool, encoder4 = encoder_block(encoder3_pool, 512) # 8
	center = conv_block(encoder4_pool, 1024) # center
	decoder4 = decoder_block(center, encoder4, 512) # 16
	decoder3 = decoder_block(decoder4, encoder3, 256) # 32
	decoder2 = decoder_block(decoder3, encoder2, 128) # 64
	decoder1 = decoder_block(decoder2, encoder1, 64) # 128
	decoder0 = decoder_block(decoder1, encoder0, 32) # 256
	outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(decoder0)

	model = models.Model(inputs=[inputs], outputs=[outputs])

	model.compile(
		optimizer=OPTIMIZER, 
    loss = weighted_bce,
		#loss=losses.get(LOSS),
		metrics=[metrics.get(metric) for metric in METRICS])

	return model


log_dir = 'drive/My Drive/Tensorflow/models/UNET256'

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    log_dir+'best_weights.hdf5',
    monitor='val_mean_io_u',
    verbose=1,
    save_best_only=True,
    mode='max'
    )

# Training the model

You train a Keras model by calling `.fit()` on it.  Here we're going to train for 10 epochs, which is suitable for demonstration purposes.  For production use, you probably want to optimize this parameter, for example through [hyperparamter tuning](https://cloud.google.com/ml-engine/docs/tensorflow/using-hyperparameter-tuning).

In [0]:
m = get_model()


m.fit(
    x=training, 
    epochs=EPOCHS, 
    steps_per_epoch=int(TRAIN_SIZE / BATCH_SIZE), 
    validation_data=evaluation,
    validation_steps=EVAL_SIZE/BATCH_SIZE,
    callbacks = [checkpoint]
    )

#We save the model definition and weights to google drive (free) 
m.save('drive/My Drive/Tensorflow/models/UNET256/UNET256.h5')

##Train from checkpoints
If we want to resume or continue training from a previous checkpoint we load the model and best weights from GDrive, check the current accuracy on the evaluation data, and resume training.

In [0]:
#bring in the architecture and best weights from Drive
m = models.load_model('drive/My Drive/Tensorflow/models/UNET256/UNET256.h5', custom_objects={'weighted_bce': weighted_bce})
m.load_weights('drive/My Drive/Tensorflow/models/UNET256/best_weights.hdf5') 

#lets see where were at
evalMetrics = m.evaluate(x=evaluation, steps = EVAL_SIZE, verbose = 1)

In [0]:
#set the monitored value (val_mean_io_u) to current evaluation output
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    log_dir+'best_weights.hdf5',
    monitor='val_mean_io_u',
    verbose=1,
    save_best_only=True,
    mode='max'
    )

checkpoint.best = evalMetrics[2]
print(checkpoint.__dict__)
print(checkpoint.best)

## Set up tensorboard

In [0]:
tensorboard = tf.keras.callbacks.TensorBoard(log_dir= 'drive/My Drive/Tensorflow/models/UNET256')

In [0]:
#Now keep training!
m.fit(
    x=training, 
    epochs= 10, 
    steps_per_epoch=int(TRAIN_SIZE / BATCH_SIZE), 
    validation_data=evaluation,
    validation_steps=EVAL_SIZE/BATCH_SIZE,
    callbacks = [checkpoint, tensorboard]
    )
#m.save('drive/My Drive/Tensorflow/models/UNET256/UNET256.h5')

In [0]:
m.save('drive/My Drive/Tensorflow/models/UNET256/UNET256.h5')

In [0]:
%tensorboard --logdir 'drive/My Drive/Tensorflow/models/UNET256'

# Prediction

The prediction pipeline is:

1.  Export imagery on which to do predictions from Earth Engine in TFRecord format to a Cloud Storge bucket.
2.  Use the trained model to make the predictions.
3.  Write the predictions to a TFRecord file in a Cloud Storage.
4.  Upload the predictions TFRecord file to Earth Engine.

The following functions handle this process.  It's useful to separate the export from the predictions so that you can experiment with different models without running the export every time.

In [0]:
#Inspect the prediction outputs
predictions = m.predict(evaluation, steps=1, verbose=1)
for prediction in predictions:
  print(predictions)

In [0]:
def doExport(image, out_image_base, kernel_buffer, region):
  """Run the image export task.  Block until complete.
  """
  task = ee.batch.Export.image.toCloudStorage(
    image = image.select(BANDS), 
    description = out_image_base, 
    bucket = BUCKET, 
    fileNamePrefix = FOLDER + '/' + PRED_BASE + '/' + out_image_base, 
    region = region.getInfo()['coordinates'], 
    scale = 30, 
    fileFormat = 'TFRecord', 
    maxPixels = 1e10,
    formatOptions = { 
      'patchDimensions': KERNEL_SHAPE,
      'kernelSize': kernel_buffer,
      'compressed': True,
      'maxFileSize': 104857600
    }
  )
  task.start()

  # Block until the task completes.
  print('Running image export to Cloud Storage...')
  import time
  while task.active():
    time.sleep(30)

  # Error condition
  if task.status()['state'] != 'COMPLETED':
    print('Error with image export.')
  else:
    print('Image export completed.')

In [0]:
def doPrediction(out_image_base, user_folder, kernel_buffer, region):
  """Perform inference on exported imagery, upload to Earth Engine.
  """

  print('Looking for TFRecord files...')
  
  # Get a list of all the files in the output bucket.
  filesList = !gsutil ls 'gs://'{BUCKET}'/'{FOLDER}'/'{PRED_BASE}
  # Get only the files generated by the image export.
  exportFilesList = [s for s in filesList if out_image_base in s]

  # Get the list of image files and the JSON mixer file.
  imageFilesList = []
  jsonFile = None
  for f in exportFilesList:
    if f.endswith('.tfrecord.gz'):
      imageFilesList.append(f)
    elif f.endswith('.json'):
      jsonFile = f

  # Make sure the files are in the right order.
  imageFilesList.sort()

  from pprint import pprint
  pprint(imageFilesList)
  print(jsonFile)
  
  import json
  # Load the contents of the mixer file to a JSON object.
  jsonText = !gsutil cat {jsonFile}
  # Get a single string w/ newlines from the IPython.utils.text.SList
  mixer = json.loads(jsonText.nlstr)
  pprint(mixer)
  patches = mixer['totalPatches']
  
  # Get set up for prediction.
  x_buffer = int(kernel_buffer[0] / 2)
  y_buffer = int(kernel_buffer[1] / 2)

  buffered_shape = [
      KERNEL_SHAPE[0] + kernel_buffer[0],
      KERNEL_SHAPE[1] + kernel_buffer[1]]

  imageColumns = [
    tf.FixedLenFeature(shape=buffered_shape, dtype=tf.float32) 
      for k in BANDS
  ]

  imageFeaturesDict = dict(zip(BANDS, imageColumns))

  def parse_image(example_proto):
    return tf.parse_single_example(example_proto, imageFeaturesDict)

  def toTupleImage(dict):
    inputsList = [dict.get(key) for key in BANDS]
    stacked = tf.stack(inputsList, axis=0)
    stacked = tf.transpose(stacked, [1, 2, 0])
    return stacked
  
   # Create a dataset from the TFRecord file(s) in Cloud Storage.
  imageDataset = tf.data.TFRecordDataset(imageFilesList, compression_type='GZIP')
  imageDataset = imageDataset.map(parse_image, num_parallel_calls=5)
  imageDataset = imageDataset.map(toTupleImage).batch(1)
  
  # Perform inference.
  print('Running predictions...')
  predictions = m.predict(imageDataset, steps=patches, verbose=1)
  # print(predictions[0])

  print('Writing predictions...')
  out_image_file = 'gs://' + BUCKET + '/' + FOLDER + '/' + PRED_BASE + '/outputs/' + out_image_base + '.TFRecord'
  writer = tf.python_io.TFRecordWriter(out_image_file)
  patches = 0
  for predictionPatch in predictions:
    print('Writing patch ' + str(patches) + '...')
    predictionPatch = predictionPatch[
        x_buffer:x_buffer+KERNEL_SIZE, y_buffer:y_buffer+KERNEL_SIZE]

    # Create an example.
    example = tf.train.Example(
      features=tf.train.Features(
        feature={
          'probability': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=predictionPatch.flatten()))
        }
      )
    )
    # Write the example.
    writer.write(example.SerializeToString())
    patches += 1

  writer.close()

  # Start the upload.
  out_image_asset = user_folder + '/' + out_image_base
  !earthengine upload image --asset_id={out_image_asset} {out_image_file} {jsonFile}

Now there's all the code needed to run the prediction pipeline, all that remains is to specify the output region in which to do the prediction, the names of the output files, where to put them, and the shape of the outputs.  In terms of the shape, the model is trained on 256x256 patches, but can work (in theory) on any patch that's big enough with even dimensions ([reference](https://www.cv-foundation.org/openaccess/content_cvpr_2015/papers/Long_Fully_Convolutional_Networks_2015_CVPR_paper.pdf)).  Because of tile boundary artifacts, give the model slightly larger patches for prediction, then clip out the middle 256x256 patch.  This is controlled with a kernel buffer, half the size of which will extend beyond the kernel buffer.  For example, specifying a 128x128 kernel will append 64 pixels on each side of the patch, to ensure that the pixels in the output are taken from inputs completely covered by the kernel.  

In [0]:
# Choose the GEE folder in which to ingest prediction image:
user_folder = 'users/defendersofwildlifeGIS/NC_solar'

# Base file name to use for TFRecord files and assets.
nc_image_base = 'nc_solar_unet256KerasPred'
# Half this will extend on the sides of each patch.
nc_kernel_buffer = [128, 128]
# NC
nc_region = nc.geometry()

In [0]:
#@title Don't run
# Run the export.
doExport(normImage, nc_image_base, nc_kernel_buffer, nc_region)

In [0]:
#@title Don't run
# Run the prediction.
doPrediction(nc_image_base, user_folder, nc_kernel_buffer, nc_region)

# Display the output

One the data has been exported, the model has made predictions and the predictions have been written to a file, and the image imported to Earth Engine, it's possible to display the resultant Earth Engine asset.  Here, display the impervious area predictions over Beijing, China.

In [0]:
out_image = ee.Image(user_folder + '/' + nc_image_base)
mapid = out_image.getMapId({'min': 0, 'max': 1})
map = folium.Map(location=[39.898, 116.5097])
map.add_ee_layer(out_image, {'min': 0, 'max': 1}, 'solar predictions')
map.add_child(folium.LayerControl())
map